# Quantum Cryptography Testing Notebook

This notebook demonstrates how to use the liboqs library for post-quantum cryptography, specifically:

1. **ML-KEM-768** (previously known as Kyber-768) - A key encapsulation mechanism (KEM) 
   selected by NIST as a post-quantum standard
   
2. **Falcon-512** - A quantum-resistant digital signature algorithm also selected 
   by NIST as a post-quantum standard

This notebook uses the direct liboqs Python bindings to demonstrate these algorithms.

In [9]:
# Import the oqs library for quantum-safe cryptography
import oqs

# Display OQS version and available algorithms
print(f"OQS Version: {oqs.oqs_version()}")
print(f"Available KEMs: {oqs.get_enabled_kem_mechanisms()[:3]}")
print(f"Available Signatures: {oqs.get_enabled_sig_mechanisms()[:3]}")

OQS Version: 0.14.1-dev
Available KEMs: ('BIKE-L1', 'BIKE-L3', 'BIKE-L5')
Available Signatures: ('Dilithium2', 'Dilithium3', 'Dilithium5')


In [10]:
# Direct test of oqs library without using service.py
import oqs

# Test ML-KEM (Kyber-768)
algorithm = "Kyber768"  # ML-KEM-768 is called Kyber768 in the API

# Generate keypair
with oqs.KeyEncapsulation(algorithm) as kem:
    public_key = kem.generate_keypair()  # This returns the public key
    secret_key = kem.export_secret_key()  # Get the private key
    
print(f"ML-KEM-768 Public Key: {len(public_key)} bytes")
print(f"ML-KEM-768 Private Key: {len(secret_key)} bytes")

# Encapsulate (create shared secret)
with oqs.KeyEncapsulation(algorithm) as kem:
    ciphertext, shared_secret = kem.encap_secret(public_key)
    
print(f"Ciphertext: {len(ciphertext)} bytes")
print(f"Shared Secret: {len(shared_secret)} bytes")

# Decapsulate (recover shared secret)
with oqs.KeyEncapsulation(algorithm, secret_key=secret_key) as kem:
    decapsulated_secret = kem.decap_secret(ciphertext)
    
print(f"Decapsulated Secret: {len(decapsulated_secret)} bytes")
print(f"Secrets match: {shared_secret == decapsulated_secret}")

# Test Falcon-512
sig_algorithm = "Falcon-512"

# Generate signature keypair
with oqs.Signature(sig_algorithm) as sig:
    sig_public_key = sig.generate_keypair()  # This returns the public key
    sig_secret_key = sig.export_secret_key()  # Get the private key
    
print(f"Falcon-512 Public Key: {len(sig_public_key)} bytes")
print(f"Falcon-512 Private Key: {len(sig_secret_key)} bytes")

# Sign a message
message = b"Test quantum signature"
with oqs.Signature(sig_algorithm, secret_key=sig_secret_key) as sig:
    signature = sig.sign(message)
    
print(f"Signature size: {len(signature)} bytes")

# Verify the signature
with oqs.Signature(sig_algorithm) as sig:
    is_valid = sig.verify(message, signature, sig_public_key)
    
print(f"Falcon-512 Signature Valid: {is_valid}")

ML-KEM-768 Public Key: 1184 bytes
ML-KEM-768 Private Key: 2400 bytes
Ciphertext: 1088 bytes
Shared Secret: 32 bytes
Decapsulated Secret: 32 bytes
Secrets match: True
Falcon-512 Public Key: 897 bytes
Falcon-512 Private Key: 1281 bytes
Signature size: 658 bytes
Falcon-512 Signature Valid: True


## Direct Test of OQS Library

Let's test the liboqs library directly without using our service.py wrapper. 
This demonstrates the core API for quantum-safe cryptography using ML-KEM-768 and Falcon-512.

In [7]:
print(dir(oqs))

['KeyEncapsulation', 'MechanismNotEnabledError', 'MechanismNotSupportedError', 'OQS_SUCCESS', 'OQS_VERSION', 'Signature', 'StatefulSignature', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'get_enabled_kem_mechanisms', 'get_enabled_sig_mechanisms', 'get_enabled_stateful_sig_mechanisms', 'get_supported_kem_mechanisms', 'get_supported_sig_mechanisms', 'get_supported_stateful_sig_mechanisms', 'is_kem_enabled', 'is_sig_enabled', 'native', 'oqs', 'oqs_python_version', 'oqs_version', 'sig_supports_context']


In [11]:
print(dir(oqs.KeyEncapsulation))

['__annotations__', '__class__', '__ctypes_from_outparam__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_b_base_', '_b_needsfree_', '_fields_', '_objects', 'alg_version', 'claimed_nist_level', 'decap_secret', 'decaps_cb', 'encap_secret', 'encaps_cb', 'export_secret_key', 'free', 'generate_keypair', 'generate_keypair_seed', 'ind_cca', 'keypair_cb', 'keypair_derand_cb', 'length_ciphertext', 'length_keypair_seed', 'length_public_key', 'length_secret_key', 'length_shared_secret', 'method_name']


## Updating Service.py

The error we encountered was that in our `service.py` file, the `ProductionMLKEM768` class was trying to use a non-existent method `export_public_key()`. 

To fix this, we need to modify the class to use the correct API. The public key is actually returned directly by the `generate_keypair()` method, so there's no need to call a separate export method.

Here's how the fixed `generate_keypair()` method should look:

```python
def generate_keypair(self) -> Tuple[bytes, bytes]:
    """Generate ML-KEM-768 keypair using liboqs"""
    with oqs.KeyEncapsulation(self.algorithm) as kem:
        public_key = kem.generate_keypair()  # This already returns the public key
        private_key = kem.export_secret_key()
        
    logger.info(f"Generated ML-KEM-768: pub={len(public_key)}B, priv={len(private_key)}B")
    return public_key, private_key
```

Similarly, the Falcon signature class might need similar adjustments.

## Exploring OQS Library Structure

Let's explore the structure of the OQS library to understand all available functions, methods, and classes.

In [12]:
import oqs
import inspect

# Get the main attributes and functions in the oqs module
print("=== OQS Module Top-Level ===")
for name in dir(oqs):
    # Skip private attributes
    if not name.startswith('_'):
        attr = getattr(oqs, name)
        attr_type = type(attr).__name__
        if inspect.isfunction(attr):
            print(f"{name}: Function")
        elif inspect.isclass(attr):
            print(f"{name}: Class")
        else:
            print(f"{name}: {attr_type}")
            
print("\n=== OQS Module Functions ===")
for name in dir(oqs):
    if not name.startswith('_') and inspect.isfunction(getattr(oqs, name)):
        func = getattr(oqs, name)
        sig = inspect.signature(func)
        print(f"{name}{sig}")

# Explore KeyEncapsulation class
print("\n=== KeyEncapsulation Class ===")
for name in dir(oqs.KeyEncapsulation):
    # Skip private attributes
    if not name.startswith('_'):
        try:
            attr = getattr(oqs.KeyEncapsulation, name)
            attr_type = type(attr).__name__
            if inspect.isfunction(attr) or inspect.ismethod(attr):
                print(f"{name}: Method")
            else:
                print(f"{name}: {attr_type}")
        except:
            print(f"{name}: [Cannot inspect]")

# Explore Signature class
print("\n=== Signature Class ===")
for name in dir(oqs.Signature):
    # Skip private attributes
    if not name.startswith('_'):
        try:
            attr = getattr(oqs.Signature, name)
            attr_type = type(attr).__name__
            if inspect.isfunction(attr) or inspect.ismethod(attr):
                print(f"{name}: Method")
            else:
                print(f"{name}: {attr_type}")
        except:
            print(f"{name}: [Cannot inspect]")

# Get all available KEM algorithms
print("\n=== Available KEM Algorithms ===")
print(oqs.get_enabled_kem_mechanisms())

# Get all available Signature algorithms
print("\n=== Available Signature Algorithms ===")
print(oqs.get_enabled_sig_mechanisms())

=== OQS Module Top-Level ===
KeyEncapsulation: Class
MechanismNotEnabledError: Class
MechanismNotSupportedError: Class
OQS_SUCCESS: int
OQS_VERSION: str
Signature: Class
StatefulSignature: Class
get_enabled_kem_mechanisms: Function
get_enabled_sig_mechanisms: Function
get_enabled_stateful_sig_mechanisms: Function
get_supported_kem_mechanisms: Function
get_supported_sig_mechanisms: Function
get_supported_stateful_sig_mechanisms: Function
is_kem_enabled: Function
is_sig_enabled: Function
native: Function
oqs: module
oqs_python_version: Function
oqs_version: Function
sig_supports_context: Function

=== OQS Module Functions ===
get_enabled_kem_mechanisms() -> 'tuple[str, ...]'
get_enabled_sig_mechanisms() -> 'tuple[str, ...]'
get_enabled_stateful_sig_mechanisms() -> 'tuple[str, ...]'
get_supported_kem_mechanisms() -> 'tuple[str, ...]'
get_supported_sig_mechanisms() -> 'tuple[str, ...]'
get_supported_stateful_sig_mechanisms() -> 'tuple[str, ...]'
is_kem_enabled(alg_name: 'str') -> 'bool'
is

In [13]:
# Let's look at an example of using KeyEncapsulation with more detail
with oqs.KeyEncapsulation("Kyber768") as kem:
    # Print all instance attributes and methods
    print("=== KeyEncapsulation Instance Attributes and Methods ===")
    for name in dir(kem):
        if not name.startswith('_'):
            try:
                attr = getattr(kem, name)
                if callable(attr):
                    print(f"{name}: Method or Callable")
                else:
                    print(f"{name}: {attr}")
            except:
                print(f"{name}: [Cannot access]")
                
# Similarly for Signature
with oqs.Signature("Falcon-512") as sig:
    # Print all instance attributes and methods
    print("\n=== Signature Instance Attributes and Methods ===")
    for name in dir(sig):
        if not name.startswith('_'):
            try:
                attr = getattr(sig, name)
                if callable(attr):
                    print(f"{name}: Method or Callable")
                else:
                    print(f"{name}: {attr}")
            except:
                print(f"{name}: [Cannot access]")

=== KeyEncapsulation Instance Attributes and Methods ===
alg_name: Kyber768
alg_version: b'https://github.com/pq-crystals/kyber/commit/28413dfbf523fdde181246451c2bd77199c0f7ff'
claimed_nist_level: 3
decap_secret: Method or Callable
decaps_cb: None
details: {'name': 'Kyber768', 'version': 'https://github.com/pq-crystals/kyber/commit/28413dfbf523fdde181246451c2bd77199c0f7ff', 'claimed_nist_level': 3, 'is_ind_cca': True, 'length_public_key': 1184, 'length_secret_key': 2400, 'length_ciphertext': 1088, 'length_shared_secret': 32, 'length_keypair_seed': 0}
encap_secret: Method or Callable
encaps_cb: None
export_secret_key: Method or Callable
free: Method or Callable
generate_keypair: Method or Callable
generate_keypair_seed: Method or Callable
ind_cca: True
keypair_cb: None
keypair_derand_cb: None
length_ciphertext: 1088
length_keypair_seed: 0
length_public_key: 1184
length_secret_key: 2400
length_shared_secret: 32
method_name: b'Kyber768'

=== Signature Instance Attributes and Methods ===
a

In [14]:
# Let's examine a specific algorithm in detail
algorithm_name = "Kyber768"  # ML-KEM-768

# Create a KeyEncapsulation instance
with oqs.KeyEncapsulation(algorithm_name) as kem:
    print(f"=== Details for {algorithm_name} ===")
    print(f"Algorithm name: {kem.method_name}")
    print(f"Version: {kem.alg_version}")
    print(f"Claimed NIST security level: {kem.claimed_nist_level}")
    print(f"Is IND-CCA: {kem.ind_cca}")
    print(f"Length of public key: {kem.length_public_key} bytes")
    print(f"Length of secret key: {kem.length_secret_key} bytes")
    print(f"Length of ciphertext: {kem.length_ciphertext} bytes")
    print(f"Length of shared secret: {kem.length_shared_secret} bytes")
    
# Let's do the same for a signature algorithm
sig_algorithm_name = "Falcon-512"

# Create a Signature instance
with oqs.Signature(sig_algorithm_name) as sig:
    print(f"\n=== Details for {sig_algorithm_name} ===")
    print(f"Algorithm name: {sig.method_name}")
    print(f"Version: {sig.alg_version}")
    print(f"Claimed NIST security level: {sig.claimed_nist_level}")
    print(f"Is EUF-CMA: {sig.euf_cma}")
    print(f"Length of public key: {sig.length_public_key} bytes")
    print(f"Length of secret key: {sig.length_secret_key} bytes")
    print(f"Maximum length of signature: {sig.length_signature} bytes")

=== Details for Kyber768 ===
Algorithm name: b'Kyber768'
Version: b'https://github.com/pq-crystals/kyber/commit/28413dfbf523fdde181246451c2bd77199c0f7ff'
Claimed NIST security level: 3
Is IND-CCA: True
Length of public key: 1184 bytes
Length of secret key: 2400 bytes
Length of ciphertext: 1088 bytes
Length of shared secret: 32 bytes

=== Details for Falcon-512 ===
Algorithm name: b'Falcon-512'
Version: b'20211101 with PQClean patches'
Claimed NIST security level: 1
Is EUF-CMA: True
Length of public key: 897 bytes
Length of secret key: 1281 bytes
Maximum length of signature: 752 bytes
